In [305]:
%pylab inline
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.font_manager import FontProperties
import collections
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [356]:
fins = ['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv',
           '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv']

# fins = ['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP003554_mouse/SRX026871_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP062407_mouse/SRX1149649_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP078005_mouse/SRX1900396_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP091889_mouse/SRX2255510_translating_ORFs.tsv',
#            '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP115915_mouse/SRX3110803_translating_ORFs.tsv']

In [357]:
cds = {}
uorfs = {}
start_codons = {}

In [358]:
for fin in fins:
    df = pd.read_table(fin, header=0, sep='\t', usecols=['ORF_ID', 'ORF_type', 'status', 'length', 'transcript_id', 'gene_id', 'gene_name', 'start_codon'])
    a = df.loc[(df['ORF_type'] == 'annotated') & (df['status'] == 'translating'), 'gene_id'].tolist()
    cds[fin] = set(a)
    b = df.loc[(df['ORF_type'] == 'super_uORF') & (df['status'] == 'translating'), 'gene_id'].tolist()
    uorfs[fin] = set(b)
    s = df.loc[(df['ORF_type'] == 'super_uORF') & (df['status'] == 'translating'), 'start_codon'].tolist()
    start_codons[fin] = s

In [ ]:
[len(cds[x]) for x in cds]

In [ ]:
joint = (cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv'] 
            & cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv']
            & cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv']
            & cds['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv'])

In [ ]:
len(joint)

In [360]:
[(x, len(uorfs[x]-cds[x]), len(cds[x]), len(uorfs[x]&cds[x]) / len(cds[x])) for x in uorfs]

[('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv',
  199,
  9520,
  0.1471638655462185),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv',
  72,
  10546,
  0.09747771666982742),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv',
  105,
  9246,
  0.08263032662773091),
 ('/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv',
  57,
  12744,
  0.2560420590081607)]

In [361]:
joint = (uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv'] 
            | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv']
            |  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv']
            |  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv'])

In [344]:
joint = (uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP003554_mouse/SRX026871_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP062407_mouse/SRX1149649_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP078005_mouse/SRX1900396_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP091889_mouse/SRX2255510_translating_ORFs.tsv']
           | uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/mouse/SRP115915_mouse/SRX3110803_translating_ORFs.tsv'])

In [366]:
valid = {x for x in joint if sum([x in uorfs[y] for y in uorfs]) > 1}

In [367]:
len(valid)

1801

In [ ]:
with open('mouse_uorf_three_samples.txt', 'w') as fout:
    for g in valid:
        fout.write('{}\n'.format(g[:g.rfind('.')]))

In [ ]:
joint = (uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP063852_human/SRX1254413_translating_ORFs.tsv']
            &  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP098789_human/SRX2536421_translating_ORFs.tsv']
            &  uorfs['/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP102021_human/SRX2647167_translating_ORFs.tsv'])

In [ ]:
len(joint)

In [ ]:
for f in start_codons:
    print(collections.Counter(start_codons[f]))

# candidate analysis

In [309]:
a = df.loc[df['ORF_type'] == 'annotated', 'gene_id'].tolist()

In [310]:
len(a)

58776

In [311]:
b = df.loc[df['ORF_type'] == 'super_uORF', 'gene_id'].tolist()

In [312]:
len(b)

77772

In [313]:
len(set(a))

22566

In [314]:
len(set(b))

12140

In [315]:
len(set(b)) / len(set(a))

0.5379774882566694

# translation analysis

In [332]:
a = df.loc[(df['ORF_type'] == 'annotated'), 'gene_id'].tolist()

In [333]:
len(a)

94421

In [334]:
b = df.loc[(df['ORF_type'] == 'super_uORF'), 'gene_id'].tolist()

In [335]:
len(b)

120224

In [336]:
len(set(a))

20412

In [338]:
len(set(b))

12775

In [339]:
len(set(b)) / len(set(a))

0.6258573388203018

In [ ]:
len(set(a) & set(b))

In [ ]:
len(set(a) - set(b))

In [ ]:
len(set(b) - set(a))

In [ ]:
s = df.loc[(df['ORF_type'] == 'super_uORF') & (df['status'] == 'translating'), 'start_codon'].tolist()

In [ ]:
collections.Counter(s)

In [331]:
fin = '/home/cmb-panasas2/wenzhenl/benchmark/uorfs/human/SRP010679_human/SRX118286_translating_ORFs.tsv'
df = pd.read_table(fin, header=0, sep='\t', usecols=['ORF_ID', 'ORF_type', 'status', 'length', 'transcript_id', 'gene_id', 'gene_name', 'start_codon'])